# Enter State Farm

In [ ]:
from theano.sandbox import cuda
cuda.use('gpu1')

In [2]:
import keras.backend as K
K.set_image_data_format('channels_first')
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "/home/ubuntu/state_farm/data/sample/"
#import utils; reload(utils)
from fastai import *
from IPython.display import FileLink
#from fastai import *


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [ ]:
import os
os.getcwd()
#os.chdir('/home/ubuntu/state_farm/data/sample/')


In [4]:
batch_size=64

## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

In [ ]:
%cd data/state

In [ ]:
%cd train

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [ ]:
from shutil import copyfile

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [ ]:
%cd ../../..

In [ ]:
%mkdir data/state/results

In [ ]:
%mkdir data/state/sample/test

## Create batches

In [5]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [ ]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [8]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

As you can see below, this training is going nowhere...

In [9]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches,steps_per_epoch=1500//batch_size,epochs=2, validation_data=val_batches, 
                 validation_steps=1000//batch_size)

Epoch 1/2
23/23 [==============================] - 17s - loss: 13.5622 - acc: 0.1195 - val_loss: 14.2479 - val_acc: 0.1160
Epoch 2/2
23/23 [==============================] - 14s - loss: 14.0912 - acc: 0.1229 - val_loss: 13.9037 - val_acc: 0.1329


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [ ]:
model.summary()

Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [ ]:
10*3*224*224

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [22]:
np.round(model.predict_generator(batches, 1500//batch_size)[:10],2)

array([[ 0.53,  0.02,  0.02,  0.01,  0.02,  0.05,  0.03,  0.12,  0.  ,  0.19],
       [ 0.01,  0.  ,  0.  ,  0.05,  0.74,  0.02,  0.01,  0.  ,  0.  ,  0.16],
       [ 0.  ,  0.  ,  0.01,  0.  ,  0.  ,  0.  ,  0.97,  0.01,  0.  ,  0.  ],
       [ 0.02,  0.  ,  0.02,  0.13,  0.71,  0.01,  0.02,  0.01,  0.  ,  0.08],
       [ 0.03,  0.05,  0.1 ,  0.03,  0.02,  0.06,  0.26,  0.09,  0.  ,  0.35],
       [ 0.1 ,  0.  ,  0.02,  0.03,  0.13,  0.03,  0.38,  0.02,  0.2 ,  0.09],
       [ 0.69,  0.  ,  0.01,  0.08,  0.08,  0.  ,  0.1 ,  0.02,  0.  ,  0.02],
       [ 0.04,  0.04,  0.02,  0.01,  0.01,  0.07,  0.65,  0.14,  0.  ,  0.02],
       [ 0.04,  0.04,  0.66,  0.02,  0.03,  0.04,  0.1 ,  0.04,  0.01,  0.02],
       [ 0.  ,  0.02,  0.35,  0.01,  0.03,  0.06,  0.32,  0.08,  0.02,  0.1 ]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [16]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, 1500//batch_size, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=1000//batch_size)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., 23, epochs=2, validation_data=<keras.pre..., validation_steps=15)`


Epoch 1/2
23/23 [==============================] - 17s - loss: 2.5185 - acc: 0.2035 - val_loss: 4.9597 - val_acc: 0.1540
Epoch 2/2
23/23 [==============================] - 15s - loss: 1.7540 - acc: 0.4021 - val_loss: 3.3501 - val_acc: 0.1392


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [17]:
model.optimizer.lr=0.001

In [18]:
model.fit_generator(batches, 1500//batch_size, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=1000//batch_size)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., 23, epochs=4, validation_data=<keras.pre..., validation_steps=15)`
  from ipykernel import kernelapp as app


Epoch 1/4
23/23 [==============================] - 18s - loss: 1.3803 - acc: 0.5904 - val_loss: 3.0691 - val_acc: 0.1377
Epoch 2/4
23/23 [==============================] - 14s - loss: 1.1499 - acc: 0.6964 - val_loss: 3.0229 - val_acc: 0.1730
Epoch 3/4
23/23 [==============================] - 14s - loss: 0.9690 - acc: 0.7660 - val_loss: 2.8882 - val_acc: 0.2046
Epoch 4/4
23/23 [==============================] - 14s - loss: 0.8435 - acc: 0.8272 - val_loss: 2.8649 - val_acc: 0.2110


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [23]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [25]:
val_res = [model.evaluate_generator(rnd_batches, 1000//batch_size) for i in range(10)]
np.round(val_res, 2)

array([[ 2.91,  0.2 ],
       [ 2.92,  0.2 ],
       [ 2.91,  0.21],
       [ 2.91,  0.2 ],
       [ 2.91,  0.2 ],
       [ 2.92,  0.2 ],
       [ 2.92,  0.21],
       [ 2.92,  0.2 ],
       [ 2.93,  0.2 ],
       [ 2.92,  0.2 ]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [32]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, steps_per_epoch=1500//batch_size, nb_epoch=2, validation_data=val_batches, 
                 validation_steps= 1000//batch_size)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_regularizer=<keras.reg...)`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=2, validation_data=<keras.pre..., steps_per_epoch=23, validation_steps=15)`


Epoch 1/2
23/23 [==============================] - 18s - loss: 5.2529 - acc: 0.2901 - val_loss: 12.2250 - val_acc: 0.1245
Epoch 2/2
23/23 [==============================] - 14s - loss: 3.7422 - acc: 0.5673 - val_loss: 10.0299 - val_acc: 0.2215


In [31]:
??model.fit_generator

In [28]:
model.optimizer.lr=0.001

In [30]:
model.fit_generator(batches, 1500//batch_size, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=1000//batch_size)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., 23, epochs=4, validation_data=<keras.pre..., validation_steps=15)`
  from ipykernel import kernelapp as app


Epoch 1/4
23/23 [==============================] - 18s - loss: 3.0474 - acc: 0.7500 - val_loss: 6.9238 - val_acc: 0.2009
Epoch 2/4
23/23 [==============================] - 15s - loss: 3.0082 - acc: 0.8030 - val_loss: 7.1103 - val_acc: 0.2226
Epoch 3/4
23/23 [==============================] - 14s - loss: 2.8771 - acc: 0.8210 - val_loss: 6.5819 - val_acc: 0.1835
Epoch 4/4
23/23 [==============================] - 15s - loss: 2.7831 - acc: 0.8399 - val_loss: 6.1925 - val_acc: 0.2379


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [ ]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [ ]:
conv1(batches)

The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Height shift: move the image up and down -

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Random shear angles (max in radians) -

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Rotation: max in degrees -

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Channel shift: randomly changing the R,G,B colors - 

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

And finally, putting it all together!

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Lucky we tried that - we starting to make progress! Let's keep going.

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.